In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
import math
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import os
import gzip
import csv
from sentence_transformers import evaluation


In [3]:
train = pd.read_csv('train_data_NLP_DucMinh.csv')
test = pd.read_csv('test_data_NLP_DucMinh.csv',encoding='windows-1252')
dev = pd.read_csv('dev_data_NLP_DucMinh.csv', encoding='windows-1252')

In [14]:
train['Quality']

0       1
1       0
2       1
3       0
4       1
       ..
3473    1
3474    0
3475    1
3476    0
3477    1
Name: Quality, Length: 3478, dtype: int64

In [16]:
train_samples = []
dev_samples = []
label_id = train['Quality']
for row in train:
    train_samples.append(InputExample(texts=[row['#1 String'],row['#2 String']],label=label_id))

TypeError: cannot convert the series to <class 'int'>

In [4]:
test['Quality']

0       1
1       1
2       1
3       0
4       0
       ..
1645    0
1646    0
1647    0
1648    1
1649    1
Name: Quality, Length: 1650, dtype: int64

In [5]:
len(train)
len(test)

1650

In [6]:
train_batch_size = 16
num_epochs = 1
model_save_path = 'output/training-' + datetime.now().strftime("%y-%m-%d_%H-%M-%S")
model_nli = CrossEncoder('distilroberta-base', num_labels=len(str(2))) # vì chỉ có 2 label là 0 và 1



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
train_dataloader = DataLoader(train, shuffle=True, batch_size=train_batch_size)

In [8]:
evaluator = CESoftmaxAccuracyEvaluator.from_input_examples(dev,name='DUC-dev')
#evaluator = evaluation.EmbeddingSimilarityEvaluator(dev['#1 String'], dev['#2 String'], dev['Quality'])
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% warm up

logging.info("Warmup steps: {}".format(warmup_steps))

# training
model_nli.fit(train_dataloader=train_dataloader, evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=10000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


AttributeError: 'str' object has no attribute 'texts'